# 1. JSON을 txt로 변환

In [ ]:
import json
import os
import glob
import ast  # 문자열을 리스트로 변환하는 모듈

# 데이터 폴더 경로
dataset_root = "C:/Code/class/DL/Feb_prj/data/data"

# Train & Validation JSON 및 이미지 폴더 경로
train_json_folder = os.path.join(dataset_root, "train/02.라벨링데이터")
train_image_folder = os.path.join(dataset_root, "train/01.원천데이터")
val_json_folder = os.path.join(dataset_root, "validation/02.라벨링데이터")
val_image_folder = os.path.join(dataset_root, "validation/01.원천데이터")

# YOLO 변환 데이터 저장 폴더
train_output_labels = os.path.join(dataset_root, "train/02.라벨링데이터")
val_output_labels = os.path.join(dataset_root, "validation/02.라벨링데이터")

# 저장할 폴더 생성
os.makedirs(train_output_labels, exist_ok=True)
os.makedirs(val_output_labels, exist_ok=True)

# **클래스 자동 수집**
unique_classes = set()

# **JSON 파일을 확인하여 모든 클래스 리스트를 자동으로 생성하는 함수**
def collect_classes(json_folder):
    json_files = glob.glob(os.path.join(json_folder, "**", "*.json"), recursive=True)
    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        annotations = data.get("Learning Data Info", {}).get("annotations", [])
        for annotation in annotations:
            class_id = annotation.get("class_id")
            unique_classes.add(class_id)  # 클래스 ID 저장

# Train & Validation 폴더에서 클래스 수집
collect_classes(train_json_folder)
collect_classes(val_json_folder)

# **자동으로 YOLO 클래스 매핑 생성**
class_mapping = {class_id: idx for idx, class_id in enumerate(sorted(unique_classes))}
print(f"✅ YOLO 클래스 매핑 완료: {class_mapping}")

# **JSON에서 실제 이미지 크기(해상도)를 가져오는 함수**
def get_image_size_from_json(json_data):
    """JSON 파일에서 이미지 해상도를 가져와 정수형으로 변환"""
    resolution = json_data.get("Raw Data Info", {}).get("resolution", "3840, 2160")
    try:
        width, height = map(int, resolution.split(", "))  # "3840, 2160" → (3840, 2160)
        return width, height
    except ValueError:
        print(f"⚠️ 해상도 값이 잘못됨: {resolution}, 기본값(3840x2160) 사용")
        return 3840, 2160  # 기본 해상도 반환

# **바운딩 박스 좌표 변환 함수**
def parse_bbox(bbox):
    """바운딩 박스 좌표를 리스트로 변환하고 값이 문자열이면 변환"""
    if isinstance(bbox, str):
        try:
            bbox = ast.literal_eval(bbox)  # 문자열을 리스트로 변환
        except (SyntaxError, ValueError):
            print(f"⚠️ 바운딩 박스 좌표 변환 실패: {bbox}")
            return None
    if len(bbox) != 4:
        print(f"⚠️ 바운딩 박스 좌표 개수 오류: {bbox}")
        return None
    return bbox

# JSON 파일을 YOLO 형식으로 변환하는 함수
def convert_json_to_yolo(json_folder, image_folder, output_labels_folder):
    json_files = glob.glob(os.path.join(json_folder, "**", "*.json"), recursive=True)
    print(f"🔍 {json_folder} 내 JSON 파일 개수: {len(json_files)}")

    for json_file in json_files:
        try:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)

            # **JSON 파일명에서 확장자 제거하여 이미지 파일명 생성**
            json_filename = os.path.basename(json_file)
            image_filename = os.path.splitext(json_filename)[0] + ".jpg"

            txt_file_path = os.path.join(output_labels_folder, os.path.splitext(json_filename)[0] + ".txt")

            # JSON에서 해상도(이미지 크기) 가져오기
            img_w, img_h = get_image_size_from_json(data)

            annotations = data.get("Learning Data Info", {}).get("annotations", [])
            yolo_labels = []

            for annotation in annotations:
                class_id = annotation.get("class_id")
                bbox = annotation.get("coord")

                if class_id not in class_mapping:
                    print(f"⚠️ 알 수 없는 클래스: {class_id}, 해당 객체는 제외됨.")
                    continue

                yolo_class_id = class_mapping[class_id]

                # **바운딩 박스 좌표 변환**
                bbox = parse_bbox(bbox)
                if bbox is None:
                    continue  # 좌표 변환 실패한 경우 해당 객체 무시

                x_min, y_min, bbox_w, bbox_h = bbox
                center_x = x_min + bbox_w / 2
                center_y = y_min + bbox_h / 2

                # YOLO 정규화 (각 이미지의 실제 해상도 기준)
                center_x /= img_w
                center_y /= img_h
                bbox_w /= img_w
                bbox_h /= img_h

                # **YOLO 형식 데이터 추가 (주석 제거)**
                yolo_label = f"{yolo_class_id} {center_x:.6f} {center_y:.6f} {bbox_w:.6f} {bbox_h:.6f}"
                yolo_labels.append(yolo_label)  # ⚠️ 주석 제거

            with open(txt_file_path, "w") as f:
                f.write("\n".join(yolo_labels))

            print(f"✅ 변환 완료: {txt_file_path} (이미지 크기: {img_w}x{img_h})")

        except Exception as e:
            print(f"⚠️ 변환 실패: {json_file} - {str(e)}")

# Train & Validation 데이터 변환 실행
convert_json_to_yolo(train_json_folder, train_image_folder, train_output_labels)
convert_json_to_yolo(val_json_folder, val_image_folder, val_output_labels)

print("🎯 모든 JSON 파일을 YOLO 포맷으로 변환 완료!")

# 2. XML을 txt로 변환

In [ ]:
import os
import xml.etree.ElementTree as ET

# 🚗 클래스 매핑
class_mapping = {
    "승용/승합": 0,
    "버스": 1,
    "화물/기타": 2,
    "이륜차": 3,
    "자전거": 4
}

def convert_xml_to_yolo(xml_path, output_dir):
    """
    XML을 YOLO 포맷으로 변환하는 함수.
    - XML 파일 하나에서 여러 개의 이미지 정보를 가져와 개별 txt 파일을 생성
    - 이미지 파일명이 jpg 형식이면 그대로 txt로 저장
    - 아니라면 XML 파일 이름 + frame_ 뒤 3자리 숫자로 txt 파일명 생성
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # XML 파일 이름에서 기본 이름 추출 
    base_name = os.path.splitext(os.path.basename(xml_path))[0]

    for image in root.findall("image"):
        img_name = image.get("name")   
        
        # 이미지 파일명이 ".jpg"로 끝나는 경우 그대로 사용, 그렇지 않으면 기존 방식 사용
        if img_name.endswith(".jpg"):
            txt_file_name = img_name.replace(".jpg", ".txt")
        else:
            img_suffix = img_name[-3:]  # frame_ 뒤의 마지막 3자리 숫자 추출 (예: "356")
            txt_file_name = f"{base_name}_{img_suffix}.txt"  # 최종 YOLO txt 파일명

        img_width = int(image.get("width"))
        img_height = int(image.get("height"))

        yolo_annotations = []

        for box in image.findall("box"):
            class_name = box.get("label")
            if class_name not in class_mapping:
                continue  # 매핑에 없는 클래스는 무시

            class_id = class_mapping[class_name]
            xtl = float(box.get("xtl"))
            ytl = float(box.get("ytl"))
            xbr = float(box.get("xbr"))
            ybr = float(box.get("ybr"))

            # YOLO 포맷 좌표 변환
            x_center = ((xtl + xbr) / 2) / img_width
            y_center = ((ytl + ybr) / 2) / img_height
            bbox_width = (xbr - xtl) / img_width
            bbox_height = (ybr - ytl) / img_height

            yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")

        # YOLO 포맷 TXT 저장 (각 이미지마다 개별 txt 파일 생성)
        output_txt_path = os.path.join(output_dir, txt_file_name)
        if yolo_annotations:
            with open(output_txt_path, "w") as f:
                f.write("\n".join(yolo_annotations))
            print(f"✅ 변환 완료: {output_txt_path}")
        else:
            print(f"⚠️ {txt_file_name}: 객체 없음, 변환 생략")


def process_all_xml(input_dir, output_dir):
    """
    입력 디렉토리에서 모든 XML 파일을 찾아 YOLO 포맷으로 변환
    """
    os.makedirs(output_dir, exist_ok=True)

    xml_files = [f for f in os.listdir(input_dir) if f.endswith(".xml")]
    if not xml_files:
        print("⚠️ 변환할 XML 파일이 없습니다.")
        return

    for xml_file in xml_files:
        xml_path = os.path.join(input_dir, xml_file)
        print(f"🔄 XML 변환 시작: {xml_path}")
        convert_xml_to_yolo(xml_path, output_dir)


# 실행 경로 설정
input_directory = "C:/Code/class/DL/Feb_prj/data/zip/val_temp/labels"  # XML 파일이 있는 폴더
output_directory = "C:/Code/class/DL/Feb_prj/data/dataset/validation/labels"  # YOLO txt 파일을 저장할 폴더

# 변환 실행
process_all_xml(input_directory, output_directory)

In [ ]:
import yaml

# 🚗 YOLO 학습을 위한 클래스 매핑
class_mapping = {
    "승용/승합": 0,
    "버스": 1,
    "화물/기타": 2,
    "이륜차": 3,
    "자전거": 4
}

# YAML 파일 생성 함수
def create_yaml_file(output_path):
    yaml_data = {
        "train": "C:/Code/class/DL/Feb_prj/data/dataset/train/images",  # 학습 데이터 경로
        "val": "C:/Code/class/DL/Feb_prj/data/dataset/validation/images",  # 검증 데이터 경로
        "nc": len(class_mapping),  # 클래스 개수
        "names": list(class_mapping.keys())  # 클래스 리스트
    }

    with open(output_path, "w", encoding="utf-8") as f:
        yaml.dump(yaml_data, f, default_flow_style=False, allow_unicode=True)

    print(f"✅ YAML 파일 생성 완료: {output_path}")

# YAML 파일 저장 경로
yaml_output_path = "C:/Code/class/DL/Feb_prj/dataset.yaml"

# YAML 파일 생성 실행
create_yaml_file(yaml_output_path)

✅ YAML 파일 생성 완료: C:/Code/class/DL/Feb_prj/dataset.yaml


# 3. 학습

In [ ]:
from ultralytics import YOLO

# 1. 사전학습된 YOLO11n 모델 불러오기
model = YOLO("yolo11n.pt")  # YOLO11n 사전학습 모델 사용

# 2. 학습 실행
model.train(
    data="space.yaml",  # 학습 데이터셋 설정 파일
    epochs=10,  # 학습 횟수 (필요에 따라 조정 가능)
    batch=16,  # 배치 크기 (메모리에 맞게 조정)
    imgsz=640,  # 입력 이미지 크기
    workers=4,  # 데이터 로딩을 위한 병렬 처리 개수
    device="cuda"  # GPU 사용 (GPU가 없으면 "cpu")
)

In [ ]:
# 3. 검증 실행 (Validation)
metrics = model.val()                                                                           
print("📊 검증 결과:", metrics)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import koreanize_matplotlib
from ultralytics import YOLO

# 모델 불러오기
model = YOLO('best_3000_xl.pt')

# 테스트할 이미지 경로
test_image = "C:\\Code\\class\\DL\\Feb_prj\\sample.jpg"

# 이미지에서 객체 탐지 수행
results = model.predict(source=test_image, save=False, conf = 0.5, imgsz=640)

# ✅ 예측 결과를 원본 이미지에 시각화하여 표시
for result in results:
    img = result.plot()  # 감지된 객체를 이미지 위에 그리기

    # OpenCV를 사용하여 이미지를 Matplotlib으로 출력
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR이므로 RGB로 변환
    plt.figure(figsize=(10, 6))
    plt.imshow(img)
    plt.axis("off")  # 축 제거
    plt.title("YOLO Detect 결과")
    plt.show()

In [ ]:
from ultralytics import YOLO

model = YOLO("best.pt")  # 이전 모델 로드

# 추가 학습 실행
model.train(
    data="C:\Code\class\DL\Feb_prj\car.yaml",
    epochs=50,  # 추가 학습 진행
    batch=16,  # 가능하면 32로 변경
    imgsz=640,
    workers=4,
    device="cuda",
    patience=10,  # 조기 종료 조건 추가
    augment=True,  # 데이터 증강 활성화
    lr0=0.005,  # 학습률 감소 적용
    lrf=0.0001,  # 최종 학습률 조정
)


# 4. Test

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLO 모델 불러오기
model = YOLO('best_3000_xl.pt')

# 입력 영상 파일 경로
input_video_path = "화면 녹화 중 2025-03-03 155001.mp4"  # 원본 영상 파일
output_video_path = "1_3000.mp4"  # 출력 영상 파일

# 영상 불러오기
cap = cv2.VideoCapture(input_video_path)

# 비디오 속성 가져오기
fps = int(cap.get(cv2.CAP_PROP_FPS))  # 초당 프레임 수
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 프레임 너비
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 프레임 높이

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # MP4 코덱 설정
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# 영상 프레임 단위 처리
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 영상 끝에 도달하면 종료

    # YOLO 객체 탐지 수행
    results = model.predict(source=frame, save=False, conf=0.5, imgsz=640)

    # 결과를 프레임에 시각화
    for result in results:
        frame = result.plot()  # 객체 감지된 박스를 프레임에 그림

    # 변환된 프레임을 새로운 영상으로 저장
    out.write(frame)

    # 시각적으로 확인하고 싶다면 주석 해제
    # cv2.imshow("YOLO Detection", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키를 누르면 종료
    #     break

# 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ 객체 탐지 완료! 저장된 파일: {output_video_path}")


In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# 모델 불러오기
model = YOLO('best_bev.pt')  # YOLO 학습된 모델 불러오기

# 입력 및 출력 영상 경로
input_video_path = "input_video.mp4"  # 입력 영상 파일
output_video_path = "output4.mp4"  # 출력 영상 파일

# 영상 불러오기
cap = cv2.VideoCapture(input_video_path)

# 비디오 속성 가져오기
fps = int(cap.get(cv2.CAP_PROP_FPS))  # 초당 프레임 수
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 프레임 너비
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 프레임 높이

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # MP4 코덱 설정
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# 영상 프레임 단위 처리
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 영상 끝에 도달하면 종료

    # YOLO 객체 탐지 수행
    results = model.predict(source=frame, save=False, conf=0.2, imgsz=1280)

    # 결과를 프레임에 시각화
    for result in results:
        frame = result.plot()  # 객체 감지된 박스를 프레임에 그림

    # 변환된 프레임을 새로운 영상으로 저장
    out.write(frame)

    # 시각적으로 확인하고 싶다면 주석 해제
    # cv2.imshow("YOLO Detection", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키를 누르면 종료
    #     break

# 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ 객체 탐지 완료! 저장된 파일: {output_video_path}")
